<a href="https://colab.research.google.com/github/VickyThumbi/3d-visualization/blob/master/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator

class RigidTransform:
    def __init__(self, rotations, translations):
        self.rotations = rotations
        self.translations = translations
        self.rotation_matrix = np.array([[np.cos(rotations[0])*np.cos(rotations[1]), 
                                         -np.sin(rotations[1])*np.sin(rotations[0])+np.cos(rotations[1])*np.sin(rotations[1])*np.cos(rotations[0]), 
                                         np.sin(rotations[1])*np.cos(rotations[0])+np.cos(rotations[1])*np.sin(rotations[1])*np.sin(rotations[0])],
                                        [np.sin(rotations[0])*np.cos(rotations[1]), 
                                         np.cos(rotations[1])*np.cos(rotations[0])+np.sin(rotations[1])*np.sin(rotations[1])*np.sin(rotations[0]), 
                                         -np.cos(rotations[1])*np.sin(rotations[0])+np.sin(rotations[1])*np.sin(rotations[1])*np.cos(rotations[0])],
                                        [-np.sin(rotations[1]), np.cos(rotations[1])*np.sin(rotations[0]), np.cos(rotations[1])*np.cos(rotations[0])]])
        self.translation_vector = np.array(translations)
        self.ddf = None

    def compute_ddf(self, size):
      # Compute the dense displacement field from the rotation matrix and translation vector
      ddf = np.zeros((size, size, size, 3))
      for i in range(size):
          for j in range(size):
              for k in range(size):
                  ddf[i, j, k] = np.matmul(self.rotation_matrix, np.array(
                      [i, j, k])) + self.translation_vector
      return ddf

    def apply(self, image):
        warped_image = np.zeros(image.shape)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                for k in range(image.shape[2]):
                    warped_coordinate = np.matmul(self.rotation_matrix, np.array([i, j, k])) + self.translation_vector
                    warped_image[i, j, k] = image[int(warped_coordinate[0]), int(warped_coordinate[1]), int(warped_coordinate[2])]
        return warped_image
    
    def warp(self, image_volume):
        # Get the dimensions of the input image volume
        nx, ny, nz = image_volume.shape
        # Get the grid coordinates for the input image volume
        x = np.linspace(0, nx-1, nx)
        y = np.linspace(0, ny-1, ny)
        z = np.linspace(0, nz-1, nz)
        # Compute the dense displacement field (DDF) if it's not already precomputed
        if self.ddf is None:
            self.ddf = self.compute_ddf((nx, ny, nz))
        # Get the warped image size and the grid coordinates for the warped image volume
        nx_warped, ny_warped, nz_warped = self.ddf.shape[:3]
        x_warped = np.linspace(0, nx_warped-1, nx_warped)
        y_warped = np.linspace(0, ny_warped-1, ny_warped)
        z_warped = np.linspace(0, nz_warped-1, nz_warped)
        # Interpolate the intensity values of the input image volume using the DDF
        interpolator = RegularGridInterpolator((x, y, z), image_volume)
        warped_image_volume = interpolator(self.ddf)
        return warped_image_volume
    def generate_random_transforms(num_transforms):
          random_transforms = []
          for i in range(num_transforms):
              random_translation = np.random.uniform(
                  -10, 10, size=(3, 1))
              random_rotation = np.random.uniform(
                  -np.pi/2, np.pi/2, size=(3, 1))
              random_transforms.append(RigidTransform(
                  random_rotation, random_translation))
          return random_transforms

    
    def interpolate(self, image, coordinate):
        x, y, z = coordinate
        x0, y0, z0 = np.floor(coordinate).astype(int)
        x1, y1, z1 = np.ceil(coordinate).astype(int)
        xd, yd, zd = x - x0, y - y0, z - z0
        intensity = (1 - xd) * (1 - yd) * (1 - zd) * image[x0, y0, z0] + \
                    xd * (1 - yd) * (1 - zd) * image[x1, y0, z0] + \
                    (1 - xd) * yd * (1 - zd) * image[x0, y1, z0] + \
                    xd * yd * (1 - zd) * image[x1, y1, z0] + \
                    (1 - xd) * (1 - yd) * zd * image[x0, y0, z1] + \
                    xd * (1 - yd) * zd * image[x1, y0, z1] + \
                    (1 - xd) * yd * zd * image[x0, y1, z1] + \
                    xd * yd * zd * image[x1, y1, z1]
        return intensity

    def compose(self, rotations, translations):
        # compute the rotation matrix for the input rotations
        rot_mat = self._compute_rot_mat(rotations)

        # compute the composed transformation
        rot_mat_composed = rot_mat.dot(self.rot_mat)
        trans_vec_composed = rot_mat.dot(self.trans_vec) + np.array(translations)
        
        # return the composed rigid transform
        return RigidTransform(rot_mat_composed, trans_vec_composed)



In [65]:
import numpy as np
import matplotlib.pyplot as plt

# Load image file "image_train00.npy"
image = np.load("image_train00.npy")

# Define ranges of translation and rotation parameters
translation_range = [(-10, 10), (-10, 10), (-10, 10)]
rotation_range = [(-np.pi, np.pi), (-np.pi, np.pi), (-np.pi, np.pi)]

# Experiment 1: Test image warping and transformation composing
# Randomly sample 3 sets of rigid transformation parameters

T1 = (np.random.uniform(low=rotation_range[0], high=rotation_range[1]),
      np.random.uniform(low=translation_range[0], high=translation_range[1]))
T2 = (np.random.uniform(low=rotation_range[0], high=rotation_range[1]),
      np.random.uniform(low=translation_range[0], high=translation_range[1]))
T3 = (np.random.uniform(low=rotation_range[0], high=rotation_range[1]),
      np.random.uniform(low=translation_range[0], high=translation_range[1]))
# Instantiate 3 objects for T1, T1⊕T2, T1⊕T2⊕T3
rigid_transform1 = RigidTransform(*T1)
rigid_transform2 = rigid_transform1.compose(*T2)
rigid_transform3 = rigid_transform2.compose(*T3)

# Compare the two warped images, between 1) using the composed transformations and 2) by applying T2 and T3 sequentially on previously warped images
image_warped_1 = rigid_transform3.warp(image)

rigid_transform_temp = RigidTransform(*T1)
image_warped_temp = rigid_transform_temp.warp(image)
rigid_transform_temp = rigid_transform_temp.compose(*T2)
image_warped_temp = rigid_transform_temp.warp(image_warped_temp)
rigid_transform_temp = rigid_transform_temp.compose(*T3)
image_warped_2 = rigid_transform_temp.warp(image_warped_temp)


AttributeError: ignored

In [60]:

# Load the image file
image = np.load("image_train00.npy")

# Define the range of translation and rotation parameters
translation_min, translation_max = -10, 10
rotation_min, rotation_max = -np.pi, np.pi

# Generate the random translation and rotation parameters
translation = np.random.uniform(translation_min, translation_max, (3,))
rotation = np.random.uniform(rotation_min, rotation_max, (3,))

# Initialize the first rigid transformation with the random parameters
rigid_transform_1 = RigidTransform(translation, rotation)

# Generate the second set of random translation and rotation parameters
translation = np.random.uniform(translation_min, translation_max, (3,))
rotation = np.random.uniform(rotation_min, rotation_max, (3,))

# Initialize the second rigid transformation with the random parameters
rigid_transform_2 = RigidTransform(translation, rotation)

# Compose the two rigid transformations to get the combined transformation
rigid_transform_composed = rigid_transform_1.compose(translation, rotation)

# Warp the image with the composed transformation
warped_image = rigid_transform_composed.warp(image)

# Plot the original and warped images
fig, axs = plt.subplots(1, 2)
axs[0].imshow(image[:, :, image.shape[2] // 2], cmap='gray')
axs[0].set_title('Original Image')
axs[1].imshow(warped_image[:, :, warped_image.shape[2] // 2], cmap='gray')
axs[1].set_title('Warped Image')
plt.show()


AttributeError: ignored

In [26]:

import numpy as np
image = np.load("image_train00.npy")


In [27]:
translation_range = (-10, 10)
rotation_range = (-np.pi/2, np.pi/2)


In [42]:
random_transforms = RigidTransform.generate_random_transforms(3)


TypeError: ignored

In [25]:
T1 = RigidTransform(np.random.uniform(rotation_range[0], rotation_range[1], size=(3, 1)), np.random.uniform(translation_range[0], translation_range[1], size=(3, 1)))
T2 = RigidTransform(np.random.uniform(rotation_range[0], rotation_range[1], size=(3, 1)), np.random.uniform(translation_range[0], translation_range[1], size=(3, 1)))
T3 = RigidTransform(np.random.uniform(rotation_range[0], rotation_range[1], size=(3, 1)), np.random.uniform(translation_range[0], translation_range[1], size=(3, 1)))

TypeError: ignored

In [ ]:
# Visualize the warped image.

plt.imshow(warped_image[:, :, 100], cmap="gray")
plt.show()

# Generate another random set of rigid transformation parameters, and apply the rigid
# transformation to the image.

transform2 = RigidTransform.generate_random_transform(
    translation_range, rotation_range)
warped_image2 = transform2.apply(image)


NameError: ignored

In [ ]:



# Visualize the warped image.

plt.imshow(warped_image2[:, :, 100], cmap="gray")
plt.show()

# Randomly sample 3 sets of rigid transformation parameters T1, T2 and T3 from above-
# defined range.

transform3 = RigidTransform.generate_random_transform(
    translation_range, rotation_range)
warped_image3 = transform3.apply(image)

FileNotFoundError: ignored